In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import os
from google.colab import files

projectID = 'finalproject-ecmms'
location = 'US'
client = bigquery.Client(project=projectID, location=location)

auth.authenticate_user()

print('authenticated')

In [ ]:
queries = {
    'andamento_profitti': """
        SELECT
          FORMAT_TIMESTAMP("%Y-%m", created_at) AS year_month,
          ROUND(SUM(oi.sale_price - p.cost)) AS profit
        FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
        JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
          ON oi.product_id = p.id
        WHERE oi.status = 'Complete'
        GROUP BY year_month
        ORDER BY year_month
    """,
    'actions': """
        SELECT sequence_number AS actions, COUNT(DISTINCT session_id) AS sessions
        FROM `bigquery-public-data.thelook_ecommerce.events`
        GROUP BY actions
        ORDER BY actions
    """,
    'browser': """
        SELECT browser, COUNT(DISTINCT ip_address) AS c
        FROM bigquery-public-data.thelook_ecommerce.events
        GROUP BY browser
        ORDER BY c DESC
    """,
    'traffic_source': """
        SELECT traffic_source, COUNT(DISTINCT ip_address) AS c
        FROM bigquery-public-data.thelook_ecommerce.events
        GROUP BY traffic_source
        ORDER BY c DESC
    """,
    'best_clients_2023': """
        SELECT u.id,
          u.first_name,
          u.last_name,
          u.email,
          ROUND(SUM(oi.sale_price)) AS totale_spesa,
          COUNT(DISTINCT o.order_id) AS count_orders_per_client
        FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
        JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
          ON o.user_id = u.id
        JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
          ON oi.order_id = o.order_id
        WHERE oi.status = 'Complete'AND oi.created_at >= TIMESTAMP('2023-01-01')
        GROUP BY u.id, u.first_name, u.last_name, u.email
        ORDER BY totale_spesa DESC
    """,
    'users_countries': """
        SELECT
          CASE
            WHEN country IN ('Germany', 'Deutschland') THEN 'Germany'
            ELSE country
          END AS country,
          city,
          COUNT(*) AS frequency
        FROM `bigquery-public-data.thelook_ecommerce.users`
        GROUP BY country, city
        ORDER BY frequency DESC
    """,
    'users_distribution_age_gender': """
        SELECT
          gender,
          COUNT(*) AS total,
          COUNT(CASE WHEN age < 18 THEN 1 END) AS under_18,
          COUNT(CASE WHEN age BETWEEN 18 AND 30 THEN 1 END) AS between_18_30,
          COUNT(CASE WHEN age BETWEEN 30 AND 50 THEN 1 END) AS between_30_50,
          COUNT(CASE WHEN age > 50 THEN 1 END) AS over_50
        FROM `bigquery-public-data.thelook_ecommerce.users`
        GROUP BY gender
    """,
    'products_profit': """
        WITH products_profit AS (
          SELECT
            p.id AS product_id,
            name AS product_name,
            category,
            ROUND(retail_price) AS retail_price,
            ROUND(cost) AS cost,
            ROUND(SUM(sale_price-cost)) AS profit
          FROM `bigquery-public-data.thelook_ecommerce.products` AS p
          JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS o
          ON p.id = o.product_id
          WHERE status NOT IN ('Returned', 'Canceled')
          GROUP BY product_id, product_name, category, retail_price, cost
        )
        SELECT *
        FROM (
          SELECT *
          FROM products_profit
          ORDER BY profit DESC
          LIMIT 5
        )
        UNION ALL
        (
          SELECT *
          FROM products_profit
          ORDER BY profit ASC
          LIMIT 5
        )
    """,
    'best_brands_profit_quantity': """
        WITH brand_sales_revenue AS (
          SELECT
            p.brand,
            COUNT(*) AS quantity,
            ROUND(SUM(oi.sale_price - p.cost)) AS revenue
          FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
          JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
            ON oi.product_id = p.id
          WHERE oi.status NOT IN ('Returned', 'Canceled')
          GROUP BY p.brand
        )
        SELECT *
        FROM (
          SELECT brand, quantity, revenue
          FROM brand_sales_revenue
          ORDER BY revenue DESC
          LIMIT 15
        )
        UNION ALL
        (
          SELECT brand, quantity, revenue
          FROM brand_sales_revenue
          ORDER BY quantity DESC
          LIMIT 15
        )
    """,
    'best_products_sales_profit': """
        SELECT
          p.id AS product_id,
          name AS product_name,
          category,
          ROUND(retail_price) AS retail_price,
          ROUND(cost) AS cost,
          COUNT(*) AS purchase_count,
          ROUND(SUM(sale_price - cost)) AS profit
        FROM `bigquery-public-data.thelook_ecommerce.products` AS p
        JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS o
        ON p.id = o.product_id
        WHERE status NOT IN ('Returned', 'Canceled')
        GROUP BY
          product_id,
          product_name,
          category,
          retail_price,
          cost
        ORDER BY purchase_count DESC
    """,
    'categories_age': """
        SELECT p.category,
          COUNT(CASE WHEN u.age < 18 THEN 1 END) AS under_18,
          COUNT(CASE WHEN u.age BETWEEN 18 AND 30 THEN 1 END) AS between_18_30,
          COUNT(CASE WHEN u.age BETWEEN 30 AND 50 THEN 1 END) AS between_30_50,
          COUNT(CASE WHEN u.age > 50 THEN 1 END) AS over_50
        FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
        JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
          ON oi.order_id = o.order_id
        JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
          ON oi.product_id = p.id
        JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
          ON o.user_id = u.id
        GROUP BY p.category
    """,
    'categories_gender': """
        SELECT p.category,
          COUNT(CASE WHEN u.gender = 'M' THEN 1 END) AS M,
          COUNT(CASE WHEN u.gender = 'F' THEN 1 END) AS F
        FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
        JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
          ON oi.order_id = o.order_id
        JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
          ON oi.product_id = p.id
        JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
          ON o.user_id = u.id
        GROUP BY p.category
    """,
    'category_profit': """
        SELECT
          p.category,COUNT(*) AS quantity,
          ROUND(SUM(o.sale_price-p.cost)) AS profit
        FROM `bigquery-public-data.thelook_ecommerce.order_items` AS o
        JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
        ON p.id = o.product_id
        WHERE status NOT IN ('Returned', 'Canceled')
        GROUP BY category
        ORDER BY profit DESC
    """,
    'disponibilita_centri': """
        SELECT product_distribution_center_id,
               COUNT(CASE WHEN sold_at IS NULL THEN 1 END) AS non_disponibili,
               COUNT(CASE WHEN sold_at IS NOT NULL THEN 1 END) AS disponibili,
               COUNT(*) AS totale,
               ROUND(((COUNT(CASE WHEN sold_at IS NOT NULL THEN 1 END) / COUNT(*)) * 100),1) AS percentuale_disponibili
        FROM `bigquery-public-data.thelook_ecommerce.inventory_items`
        GROUP BY product_distribution_center_id
        ORDER BY percentuale_disponibili ASC
    """,
    'disponibilita_prodotti': """
        SELECT product_id,
          COUNT(CASE WHEN sold_at IS NULL THEN 0 END) AS non_disponibili,
          COUNT(CASE WHEN sold_at IS NOT NULL THEN 1 END) AS disponibili,
          COUNT(*) AS totale,
          ROUND(((COUNT(CASE WHEN sold_at IS NOT NULL THEN 1 END) / COUNT(*)) * 100),1) AS percentuale_disponibili
        FROM `bigquery-public-data.thelook_ecommerce.inventory_items`
        GROUP BY product_id
        ORDER BY disponibili ASC
    """
}

for query_name, query in queries.items():
    query_job = client.query(query)
    results = query_job.result().to_dataframe()
    filename = f'{query_name}.csv'
    results.to_csv(filename, index=False)
    files.download(filename)

print('Download completato')
